# 6.1 Reading and Writing Data in Text Format 

There are lots of functions in pandas for reading data in  a 
text format. These functions also have lots of arguments, as
particular tasks require particular reading setups.

Let's start with a small csv:

In [2]:
!cat ../pydata-book/examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

As it is a csv, we can use `pandas.read_csv`:

In [1]:
import pandas as pd
import numpy as np
from pandas._libs import index

In [4]:
df = pd.read_csv("../pydata-book/examples/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Not all files have a header. Consider this file:

In [5]:
!cat ../pydata-book/examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Here, we have two options: let Pandas assign column names
with `header=None`, or add them ourselves with `names=[names]`

In [7]:
pd.read_csv("../pydata-book/examples/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
pd.read_csv("../pydata-book/examples/ex2.csv", names=["a", 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose we **want a specif column, such as message, as the index**.
We can do that indicating either the integer 4 or the string "message"
in the `index_col` argument:

In [9]:
names=["a", 'b', 'c', 'd', 'message']
pd.read_csv("../pydata-book/examples/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


The `index_col` argument accepts a list of indexes, as to make
a hierarchical index dataset: 

In [10]:
!cat ../pydata-book/examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [12]:
parsed = pd.read_csv("../pydata-book/examples/csv_mindex.csv", index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

Sometimes the delimiter is different than a comma. Consider
the following file:

In [13]:
!cat ../pydata-book/examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


Although one could try editing it by hand, we can pass an expression,
such as `\s+`, which will account for any number of whitespaces:

In [14]:
result = pd.read_csv("../pydata-book/examples/ex3.txt", sep="\s+")
result

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_24880/2672034830.py:1: SyntaxWarning: invalid escape sequence '\s'
  result = pd.read_csv("../pydata-book/examples/ex3.txt", sep="\s+")


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


... as the first line of this dataset has one fewer values than the
rest, pandas infers the first column to be the index.

One example of useful parsing argument is the `skiprows`, with accepts
a list of rows to skip:

In [1]:
!cat ../pydata-book/examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [4]:
pd.read_csv("../pydata-book/examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


*Handling missing data* is a point of interest. Missing data is usually 
represented in the csv file as a placeholder, such as NULL or NONE, or 
simply an empty string. Pandas uses a default set of common ocurring 
sentinels to handle missing data:

In [5]:
!cat ../pydata-book/examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

this output has the NA and empty string as non-present values.

In [6]:
result = pd.read_csv("../pydata-book/examples/ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


We can disable the default values set as NA with the `keep_default_na=false`
argument: 

In [8]:
result2 = pd.read_csv("../pydata-book/examples/ex5.csv", keep_default_na=False)
result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [9]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [10]:
result3 = pd.read_csv("../pydata-book/examples/ex5.csv", keep_default_na=False, na_values=["NA"])
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


At last, we can specify different sentinels for each column:

In [11]:
sentinels = {"message" : ["foo", 'NA'], 'something' : ['two']}
pd.read_csv('../pydata-book/examples/ex5.csv', keep_default_na=False, na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


## Reading Text Files in Pieces

To process very large files, like perhaps those related 
to biological data, we may wish to read smaller parts of 
the file or to iterate through it in smaller chunks.

To do so, we may make pandas display settings more compact:

In [10]:
pd.options.display.max_rows = 10

Now we have:

In [3]:
result = pd.read_csv("../pydata-book/examples/ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


To read a smaller number of lines, we may indicate it with
`nrows` argument: 

In [4]:
pd.read_csv("../pydata-book/examples/ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


To read iterate a file in pieces, we use the `chunksize` argument:

In [12]:
chunker = result = pd.read_csv("../pydata-book/examples/ex6.csv", chunksize=1000)
type(chunker)

pandas.io.parsers.readers.TextFileReader

This is an iterable. Let's say we want to iterate through it counting
the amount of times any given key appears:

In [13]:
tot = pd.Series([], dtype='int64')

for piece in chunker:
  tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

The `TextFileReader` object also has a `get_chunk` method that enables you
to read pieces of an arbitrary size.

## Writing Data to Text Format

This is how we export data to a delimited format. Let's consider one of the files
previously used in our examples:


In [2]:
data = pd.read_csv("../pydata-book/examples/ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


To export it with a custom delimiter:

In [3]:
data.to_csv('../pydata-book/examples/out.csv', sep='|')

In [4]:
!cat '../pydata-book/examples/out.csv'

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


By default, empty values appear as empty strings. If
we wish to use another sentinel value, `na_rep` comes 
to the rescue!

In [5]:
import sys # we'll write to sys.stdout to avoiding writing in a file
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


By default, column and row labels are written. We
can also disable this feature:

In [6]:
data.to_csv(sys.stdout, index=False, header=False) 

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


You may also write a subset of the columns in a chosen order:

In [7]:
data.to_csv(sys.stdout, index=False, columns=['c', 'message', 'd']) 

c,message,d
3.0,,4
,world,8
11.0,foo,12


## JSON data

Python has a json library built-in which we will use to read an
file example:

In [8]:
obj = """ {"name": "Wes",  "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],  "pet": null,  "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},  {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}] } """

In [9]:
import json

result = json.loads(obj)

result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

`json.dumps()` is the method for converting from python object
to json.

how to convert a json to DataFrame is up to the user. One convenient
way would be to pass a list of dictionaries (which where JSON) to the
DataFrame constructor and a subset of data fields:

In [10]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'hobbies'])
siblings

,name,hobbies
0,Scott,"[guitars, soccer]"
1,Katie,"[diving, art]"


pandas has a `read_json` method for automatically converting JSON 
datasets, although it should be a "well-behaved" json dataset. In the
following example we can see that the `read_json` method assumes each 
json object is a row in a dataset:

In [11]:
!cat ../pydata-book/examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [12]:
data = pd.read_json("../pydata-book/examples/example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


To convert back to json, pandas also has a `to_json()` method:

In [13]:
data.to_json(sys.stdout)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [14]:
data.to_json(sys.stdout, orient='records')

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

## XML and HTML: Web Scraping

Python has many libraries for reading and writing data in HTML 
and XML formats. Pandas leverages these libraries to make it 
possible to efficiently extract data from these formats with 
the `read_html()` method. It has many options, but it's default
behavior is to search and fetch data inside `<table>` tags.

Let's analyse an example provided by the book:

In [15]:
tables = pd.read_html("../pydata-book/examples/fdic_failed_bank_list.html")
len(tables)

1

In [17]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


As we will learn in the next chapters, from here it is possible to do some 
data cleaning and analysis, for example, finding the count of bank failures
per year:

In [18]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

Closing Date
2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: count, dtype: int64

Pandas also has the `read_xml()` function, which makes reading xml files
a lot easier than the alternative with the `lxml` library. The example
table is present in page 192 of the book.

## Summary

Today we learned the main methods for reading and writing text files from
and to the disk. These include:

- The `read_csv()` function, with arguments to:
    - label columns with `names` and indexes with `index_col`;
    - skiprows, skipfooter, header presence;
    - read by chunks in an iterable with `chunksize` and by pieces with `nrows`;
    - set values to consider as `NaN` with (na_values) and override the default values; 
    - set delimiter characters, including "any amount of whitespace" (`\s+`)
    - exporting with `to_csv` setting delimeters and NaN expression with `na_rep`
- The `read_json()` function and `to_json`
- The `read_html` and `read_xml` functions

**upnext**: dealing with binary formats of data, dealing with web APIs
and Databases!
    
# 6.2 Binary Data Formats

Python's built-in way to deal with binary files is the `pickle` format.
Pandas has ways of reading and writing to pickle, but that is not common 
practice due to the unstable nature of the pickle format. 

Pandas can also read other binary formats, such as HDF5, ORC, and Apache 
Parquet. `read_parquet` will be useful when dealing with pyarrow in the 
future.

## Reading Microsoft Excel Files

Reading XLS and XLSX files with pandas requires the packages `xlrd` and 
`openpyxl` as dependencies. There are two ways to handle Excel files:
The `.read_excel()` method and the `pd.ExcelFile` class.

The `ExcelFile` class allows you to access a wide variety of data in the
file, such as its sheets with `sheet_names`, and turning a sheet into a 
`DataFrame` with `parse(sheet_name='')`:

In [1]:
import pandas as pd
xlsx = pd.ExcelFile('../pydata-book/examples/ex1.xlsx')
xlsx.sheet_names 

['Sheet1']

In [2]:
xlsx.parse(sheet_name='Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


This table has a index columns which was parsed as an `Unnamed` column.
We can indicate that with `index_col` argument:

In [3]:
xlsx.parse(sheet_name='Sheet1', index_col=0 )

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


If we're dealing with only one sheet of a document, we may wish to 
load it at once with `.read_excel`:

In [4]:
frame = pd.read_excel('../pydata-book/examples/ex1.xlsx', sheet_name='Sheet1', index_col=0)
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### Writing to Excel

To write to excel, we first create an `ExcelWriter` object, which takes the 
output file as a parameter in it's creation. Then, we call `frame.to_excel()`
passing the writer object as the first parameter and the sheet name.

In [6]:
writer = pd.ExcelWriter("../pydata-book/examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.close()

/tmp/ipykernel_238093/3153009253.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  frame.to_excel(writer, "Sheet1")


We can skip the writer by simply passing a file path to `to_excel()`

In [7]:
frame.to_excel('../pydata-book/examples/ex2.xlsx')

## Using HDF5 file format

Hierarchical Data Format files are efficient in their memory usage, as
they are easily compressed. They are a good option to dealing with 
data that doesn't fit in memory.

To use HDF5 in pandas we need the `PyTables` package. Pandas then provides
a high-level interface to this library with the `pd.HDFStore` class:


In [9]:
import numpy as np
frame = pd.DataFrame({'a': np.random.standard_normal(100)})
store = pd.HDFStore('../pydata-book/examples/mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: ../pydata-book/examples/mydata.h5

We retrieve the objects stored in the HDF5 file with a dictionary-like API:

In [11]:
store['obj1_col']

0    -0.644685
1    -1.032746
2    -0.319061
3     1.317558
4     0.115972
        ...   
95    1.391957
96    0.564853
97    0.272688
98   -0.140289
99    0.525039
Name: a, Length: 100, dtype: float64

A tool like HDF5 can really speed up the process of analysis with large 
dataframes.

# 6.3 Interacting with Web APIs

We can retrieve data provided by an API (usually in JSON format) with the
`requests` package. In the example below we make a GET HTTP request of the
last 30 GitHub issues for pandas on GitHub:

In [12]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp.raise_for_status()
resp

<Response [200]>

It's good practice to call `raise_for_status()` after a get request
to check for HTTP errors.

The `response` object has a `json()` method to parse the JSON data as 
a dict or a list:

In [13]:
data = resp.json()

data[0]['title']

'BUG: preserve StringDtype in DataFrame.where with list-like other'

Each element in `data` is a dictionary with the data found on the 
Issues page, except for the comments. We can pass that data to a 
pandas DataFrame for data analysis:

In [14]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues

,number,title,labels,state
0,63874,BUG: preserve StringDtype in DataFrame.where w...,[],open
1,63872,BUG: Correct length_of_indexer for range objects,[],open
2,63870,CI: Test editable install for doctests,[],open
3,63869,DOC: minor alignment fixes to `pandas.melt()` ...,[],open
4,63868,BUG: Clarify KeyError for missing groups in Gr...,[],open
5,63866,Pr02 fix,[],open
6,63863,ENH: Data Entropy & Information Loss Tracking ...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
7,63861,DOC: define py:type targets for typing aliases,[],open
8,63860,DOC: Update DataFrame.stack() docstring for 3....,[],open
9,63859,ENH: Pass arguments to pa.to_pandas in `read_c...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open


The main idea to takeaway is that we can create interfaces to 
extract and interact with web APIs to posterior analysis with
`DataFrames`.

# 6.4 Interacting With Databases

In business, a lot of the times data is stored in a database 
which we query to interact with. Pandas has a built-in way
of interacting with such databases, but we'll demonstrate
the hurdle that is to do it 'manually', too. Consider the
database created with SQLite below:

In [15]:
import sqlite3
query = """
  CREATE TABLE test
  (a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER);
"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

We'll insert a few rows of data:


In [16]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = 'INSERT INTO test VALUES (?, ?, ?, ?)'

con.executemany(stmt, data)
con.commit() 

Usually a list of tuples is returned from a query
with a python SQL driver:

In [17]:
cursor = con.execute('SELECT * FROM test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

We can pass this list of tuples to the DataFrame constructor,
but we'll need the column names too. For `sqlite3`, this information
is contained in the cursor's `description` attribute:

In [18]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [19]:
pd.DataFrame(rows, columns=[row[0] for row in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


## The pandas way of retrieving data from databases

To do it the built-in way, we must use an `SQLAlchemy` connection:

In [21]:
import sqlalchemy as sqla

db = sqla.create_engine('sqlite:///mydata.sqlite')

pd.read_sql('SELECT * FROM test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


# Summary

Retrieving data is an important first step in data analysis.
We've been exposed to various offered by pandas to do it, 
so that we're able to easily retrieve data from *csv, xml,
html, web APIs, databases, json, excel, binary file formats*,
and more. Next, we'll start data wrangling, visualization, 
time series analysis and more.